In [ ]:
!pip install duckdb --pre --upgrade

In [ ]:
sf = 10

In [3]:
%%time

import duckdb
import pathlib
for x in range(0, sf) :
  con=duckdb.connect()
  con.sql('PRAGMA disable_progress_bar;SET preserve_insertion_order=false')
  con.sql(f"CALL dbgen(sf={sf} , children ={sf}, step = {x})") 
  for tbl in ['nation','region','customer','supplier','lineitem','orders','partsupp','part'] :
     pathlib.Path(f'/lakehouse/default/Files/{sf}/{tbl}').mkdir(parents=True, exist_ok=True) 
     con.sql(f"COPY (SELECT * FROM {tbl}) TO '/lakehouse/default/Files/{sf}/{tbl}/{x:03d}.parquet' ")
  con.close()

StatementMeta(, 0b346e6f-8846-4439-90cb-bdbb2139ac01, 5, Finished, Available)

CPU times: user 5min 56s, sys: 28.8 s, total: 6min 24s
Wall time: 2min 48s


In [5]:
from pyspark.sql.types import *
spark.conf.set("spark.sql.parquet.vorder.enabled", "true")
spark.conf.set("spark.microsoft.delta.optimizeWrite.enabled", "true")
def loadFullDataFromSource(table_name):
    df = spark.read.parquet(f'Files/{sf}/' + table_name + '/*.parquet')
    df.write.mode("overwrite").format("delta").save("Tables/" + table_name)
full_tables = [
    'customer',
    'lineitem',
    'nation',
    'orders' ,
    'region',
    'partsupp',
    'supplier' ,
    'part'
    ]

for table in full_tables:
    loadFullDataFromSource(table)

StatementMeta(, 0b346e6f-8846-4439-90cb-bdbb2139ac01, 7, Finished, Available)